# CMS ex situ GIWAXS 2023C2

# CMS GIWAXS raw data processing & exporting notebook
In this notebook you output xr.DataSets stored as .zarr stores containing all your raw,
remeshed (reciprocal space), and caked CMS GIWAXS data. Saving as a zarr automatically converts the array to a dask array

In [ ]:
# # Outdated, this used to work to just overwrite existing PyHyper install in JupyterHub conda environment
# # If you need a custom PyHyper version install, you may need your own conda environment

# # Kernel updates if needed, remember to restart kernel after running this cell!:
# !pip install -e /nsls2/users/alevin/repos/PyHyperScattering  # to use pip to install via directory

## Imports

In [ ]:
### Imports:
import pathlib
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import xarray as xr
import PyHyperScattering as phs
import pygix
import gc
from tqdm.auto import tqdm  # progress bar loader!

print(f'Using PyHyperScattering Version: {phs.__version__}')

## Defining some objects

### Define & check paths

In [ ]:
# I like pathlib for its readability & checkability, it's also necessary for the loadSeries function later on
# Replace the paths with the ones relevant to your data, you can use the ".exists()" method to make sure you defined a path correctly
userPath = pathlib.Path('/nsls2/users/alevin')
propPath = pathlib.Path('/nsls2/data/cms/proposals/2023-3/pass-311415')
dataPath = propPath.joinpath('AL_processed_data/KWhite2/waxs')
rawPath = dataPath.joinpath('raw')
samplesPath = dataPath.joinpath('stitched')
calibPath = rawPath.joinpath('AgBH_cali_5m_12.7kev_x0.000_th0.000_10.00s_1307208_waxs.tiff')
maskponiPath = propPath.joinpath('AL_processed_data/maskponi')  # place for pyhyper-drawn masks and poni files

outPath = propPath.joinpath('AL_processed_data')

# Select poni & mask filepaths
poniFile = maskponiPath.joinpath('CeO2_2023-12-03_y673_x464p3.poni')
# maskFile = maskponiPath.joinpath('blank.json')
maskFile = maskponiPath.joinpath('pilatus1m_vertical_gaps_only.json')

# Colormap
cmap = plt.cm.turbo
cmap.set_bad('black')

### Define metadata naming scheme & initialize loaders

In [ ]:
[f.name for f in sorted(samplesPath.glob('*pos1*'))]

In [ ]:
[len(f.name.split('_')) for f in sorted(samplesPath.glob('*pos1*'))]

In [ ]:
[f.name for f in sorted(samplesPath.glob('*pos1*')) if len(f.name.split('_'))==9]

In [11]:
fixed_rpm_set = [f for f in sorted(samplesPath.glob('*')) if len(f.name.split('_'))==9]
variable_rpm_set = [f for f in sorted(samplesPath.glob('*')) if len(f.name.split('_'))==10]

In [12]:
len(fixed_rpm_set)

112

In [13]:
len(variable_rpm_set)

64

In [14]:
# set ex situ metadata filename naming schemes:
fixed_rpm_md_naming_scheme =    ['project', 'material', 'solvent', 'detector_pos', 'sample_pos', 
                                 'incident_angle', 'exposure_time', 'scan_id', 'detector']
variable_rpm_md_naming_scheme = ['project', 'material', 'solvent', 'rpm', 'detector_pos', 'sample_pos', 
                                 'incident_angle', 'exposure_time', 'scan_id', 'detector']

# A way to check our naming schemes to make sure they're right:
delim = '_'
file_sets =    [            fixed_rpm_set,               variable_rpm_set]
file_schemes = [fixed_rpm_md_naming_scheme, variable_rpm_md_naming_scheme]

for file_set, file_scheme in zip(file_sets, file_schemes):
    first_filename = sorted(file_set)[0].name
    print(f'\nFilename: {first_filename}')
    first_filename_list = first_filename.split(delim)
    for tup in zip(file_scheme, first_filename_list):
        print(tup)


Filename: AL_PM6-Y6BO_CBCN_pos1_x-0.000_th0.080_10.00s_1306661_wax.tiff
('project', 'AL')
('material', 'PM6-Y6BO')
('solvent', 'CBCN')
('detector_pos', 'pos1')
('sample_pos', 'x-0.000')
('incident_angle', 'th0.080')
('exposure_time', '10.00s')
('scan_id', '1306661')
('detector', 'wax.tiff')

Filename: AL_Y6BO_CB_2000_pos1_x-0.000_th0.080_10.00s_1306221_wax.tiff
('project', 'AL')
('material', 'Y6BO')
('solvent', 'CB')
('rpm', '2000')
('detector_pos', 'pos1')
('sample_pos', 'x-0.000')
('incident_angle', 'th0.080')
('exposure_time', '10.00s')
('scan_id', '1306221')
('detector', 'wax.tiff')


In [15]:
# Initalize CMSGIWAXSLoader objects with the above naming schemes
fixed_rpm_loader = phs.load.CMSGIWAXSLoader(md_naming_scheme=fixed_rpm_md_naming_scheme)
variable_rpm_loader = phs.load.CMSGIWAXSLoader(md_naming_scheme=variable_rpm_md_naming_scheme)

## Data processing
Break this section up however makes sense for your data

### Variable RPM file set

#### intialize integrators

In [16]:
variable_rpm_recip_integrator = phs.integrate.PGGeneralIntegrator(geomethod = 'ponifile',
                                                                  ponifile = poniFile,
                                                                  output_space = 'recip')
variable_rpm_caked_integrator = phs.integrate.PGGeneralIntegrator(geomethod = 'ponifile',
                                                                  ponifile = poniFile,
                                                                  output_space = 'caked')

#### generate, check, save: recip Dataset

In [17]:
# Use the single_images_to_dataset utility function to pygix transform all raw files in an indexable list
# Located in the IntegrationUtils script, CMSGIWAXS class:

# Initalize CMSGIWAXS util object
util = phs.util.IntegrationUtils.CMSGIWAXS(sorted(variable_rpm_set), variable_rpm_loader, variable_rpm_recip_integrator)
raw_DS, recip_DS = util.single_images_to_dataset()  # run function 
display(recip_DS)

/nsls2/users/alevin/repos/PyHyperScattering/src/PyHyperScattering/IntegrationUtils.py:242: UserWarning: No mask defined. Creating an empty mask with dimensions (1073, 981).
  integ_DA = self.integrator.integrateSingleImage(DA)


Transforming Raw Data:   0%|          | 0/63 [00:00<?, ?it/s]

<xarray.Dataset>
Dimensions:  (q_z: 1073, q_xy: 981)
Coordinates:
  * q_z      (q_z) float64 -1.647 -1.643 -1.639 -1.635 ... 2.613 2.617 2.621
  * q_xy     (q_xy) float64 -1.912 -1.908 -1.904 -1.9 ... 2.1 2.104 2.108 2.112
Data variables: (12/64)
    1306221  (q_z, q_xy) float32 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    1306222  (q_z, q_xy) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    1306223  (q_z, q_xy) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    1306224  (q_z, q_xy) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    1306229  (q_z, q_xy) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    1306230  (q_z, q_xy) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    ...       ...
    1306357  (q_z, q_xy) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    1306358  (q_z, q_xy) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    1306363  (q_z, q_xy) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    1306364  (q_z, q_xy) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    1306365  (q_z, q_xy) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    1306366  (q_z, q_xy) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0

In [ ]:
# # Example of a quick plot check if desired here:
# for DA in tqdm(list(recip_DS.data_vars.values())[::8]):   
#     cmin = DA.quantile(0.01)
#     cmax = DA.quantile(0.99)
    
#     ax = DA.sel(q_xy=slice(-1.1, 2.1), q_z=slice(-0.05, 2.4)).plot.imshow(cmap=cmap, norm=plt.Normalize(cmin, cmax), figsize=(8,4))
#     ax.axes.set(aspect='equal', title=f'{DA.material}, incident angle: {DA.incident_angle}, scan id: {DA.scan_id}')
#     plt.show()
#     plt.close('all')

In [ ]:
# # Saving dataset with xarray's to_zarr() method:
# # General structure below:

# # Set where to save file and what to name it
# savePath = outPath.joinpath('testing_zarrs')
# savePath.mkdir(exist_ok=True)
# savename = 'custom_save_name.zarr'

# # Save it
# recip_DS.to_zarr(savePath.joinpath(savename))

#### generate, check, save: caked Dataset

In [18]:
# Use the single_images_to_dataset utility function to pygix transform all raw files in an indexable list
# Located in the IntegrationUtils script, CMSGIWAXS class:

# Initalize CMSGIWAXS util object
util = phs.util.IntegrationUtils.CMSGIWAXS(sorted(variable_rpm_set), variable_rpm_loader, variable_rpm_caked_integrator)
raw_DS, caked_DS = util.single_images_to_dataset()  # run function 
display(caked_DS)

/nsls2/users/alevin/repos/PyHyperScattering/src/PyHyperScattering/IntegrationUtils.py:242: UserWarning: No mask defined. Creating an empty mask with dimensions (1073, 981).
  integ_DA = self.integrator.integrateSingleImage(DA)


Transforming Raw Data:   0%|          | 0/63 [00:00<?, ?it/s]

<xarray.Dataset>
Dimensions:  (chi: 180, qr: 1000)
Coordinates:
  * chi      (chi) float64 -89.5 -88.5 -87.5 -86.5 -85.5 ... 86.5 87.5 88.5 89.5
  * qr       (qr) float64 0.0009973 0.004259 0.007521 ... 3.253 3.256 3.26
Data variables: (12/64)
    1306221  (chi, qr) float32 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    1306222  (chi, qr) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    1306223  (chi, qr) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    1306224  (chi, qr) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    1306229  (chi, qr) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    1306230  (chi, qr) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    ...       ...
    1306357  (chi, qr) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    1306358  (chi, qr) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    1306363  (chi, qr) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    1306364  (chi, qr) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    1306365  (chi, qr) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    1306366  (chi, qr) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0

In [ ]:
# Example of a quick plot check if desired here:
for DA in tqdm(list(caked_DS.data_vars.values())[::8]):   
    cmin = DA.quantile(0.01)
    cmax = DA.quantile(0.99)
    
    ax = DA.plot.imshow(cmap=cmap, norm=plt.Normalize(cmin, cmax), figsize=(8,4))
    ax.axes.set(title=f'{DA.material}, incident angle: {DA.incident_angle}, scan id: {DA.scan_id}')
    plt.show()
    plt.close('all')

In [ ]:
# # Saving dataset with xarray's to_zarr() method:
# # General structure below:

# # Set where to save file and what to name it
# savePath = outPath.joinpath('testing_zarrs')
# savePath.mkdir(exist_ok=True)
# savename = 'custom_save_name.zarr'

# # Save it
# caked_DS.to_zarr(savePath.joinpath(savename))

### Fixed RPM file set

In [ ]:
# would be same code as above for another file set

## Not fully implemented

In [ ]:
def poni_centers(poniFile, pix_size=0.000172):
    """
    Returns poni center value and the corresponding pixel position. Default pixel size is 172 microns (Pilatus 1M)
    
    Inputs: poniFile as pathlib path object to the poni file
    Outputs: ((poni1, y_center), (poni2, x_center))
    """
    
    with poniFile.open('r') as f:
        lines = list(f.readlines())
    poni1_str = lines[6]
    poni2_str = lines[7]

    poni1 = float(poni1_str.split(' ')[1])
    poni2 = float(poni2_str.split(' ')[1])

    y_center = poni1 / pix_size
    x_center = poni2 / pix_size
        
    return ((poni1, y_center), (poni2, x_center))

poni_y, poni_x = poni_centers(poniFile)
display(poni_y)
display(poni_x)

### Yoneda check:
This can be used as a way to verify / refine your correct beam center y position. The yoneda peak should always appear at a q value corresponding to your incident angle plus your film's critical angle:

In [ ]:
def yoneda_qz(wavelength, alpha_crit, alpha_incidents):
    """Calculate the yoneda qz values given the wavelength, critical angle, and incident angle (in degrees)"""
    qz_inv_meters = ((4 * np.pi) / (wavelength)) * (np.sin(np.deg2rad((alpha_incidents + alpha_crit)/2)))
    qz_inv_angstroms = qz_inv_meters / 1e10
    return qz_inv_angstroms


wavelength = 9.762535309700809e-11  # 12.7 keV
alpha_crit = 0.11  # organic film critical angle
alpha_incidents = np.array([0.08, 0.1, 0.12, 0.15])  # incident angle(s)

yoneda_angles = alpha_incidents + alpha_crit

yoneda_qz(wavelength, alpha_crit, alpha_incidents)  # expected yoneda qz positions

In [ ]:
def select_attrs(data_arrays_iterable, selected_attrs_dict):
    """
    Selects data arrays whose attributes match the specified values.

    Parameters:
    data_arrays_iterable: Iterable of xarray.DataArray objects.
    selected_attrs_dict: Dictionary where keys are attribute names and 
                         values are the attributes' desired values.

    Returns:
    List of xarray.DataArray objects that match the specified attributes.
    """    
    sublist = list(data_arrays_iterable)
    
    for attr_name, attr_values in selected_attrs_dict.items():
        sublist = [da for da in sublist if da.attrs[attr_name] in attr_values]
                
    return sublist

In [ ]:
# 2D reciprocal space cartesian plots
qxy_min = -1.1
qxy_max = 2.1
qz_min = -0.01
qz_max = 2.2

selected_attrs_dict = {'material': ['PM6'], 'solvent': ['CBCN']}
# selected_attrs_dict = {}

selected_DAs = select_attrs(fixed_recip_DS.data_vars.values(), selected_attrs_dict)
for DA in tqdm(selected_DAs):
    # Slice data for selected q ranges (will need to rename q_xy if dimensions are differently named)
    sliced_DA = DA.sel(q_xy=slice(qxy_min, qxy_max), q_z=slice(qz_min, qz_max))
    
    real_min = float(sliced_DA.compute().quantile(0.05))
    cmin = 1 if real_min < 1 else real_min

    cmax = float(sliced_DA.compute().quantile(0.997))   
    
    # Plot
    ax = sliced_DA.plot.imshow(cmap=cmap, norm=plt.Normalize(cmin, cmax), interpolation='antialiased', figsize=(5.5,3.3))
    ax.colorbar.set_label('Intensity [arb. units]', rotation=270, labelpad=15)
    # ax.axes.set(aspect='equal', title=f'Cartesian Plot: {DA.material} {DA.solvent} {DA.rpm}, {float(DA.incident_angle[2:])}° Incidence',
    #             xlabel='q$_{xy}$ [Å$^{-1}$]', ylabel='q$_z$ [Å$^{-1}$]')
    ax.axes.set(aspect='equal', title=f'Cartesian Plot: {DA.material} {DA.solvent}, {float(DA.incident_angle[2:])}° Incidence',
                xlabel='q$_{xy}$ [Å$^{-1}$]', ylabel='q$_z$ [Å$^{-1}$]')
    ax.figure.set(tight_layout=True, dpi=130)
    
    # ax.figure.savefig(savePath.joinpath(f'{DA.material}-{DA.solvent}-{DA.rpm}_qxy{qxy_min}to{qxy_max}_qz{qz_min}to{qz_max}_{DA.incident_angle}.png'), dpi=150)
    # ax.figure.savefig(savePath.joinpath(f'{DA.material}-{DA.solvent}_qxy{qxy_min}to{qxy_max}_qz{qz_min}to{qz_max}_{DA.incident_angle}.png'), dpi=150)

    plt.show()
    plt.close('all')

In [ ]:
# Yoneda peak linecut check
qxy_min = 0.22
qxy_max = 2
qz_min = -0.02
qz_max = 0.06

selected_DAs = select_attrs(fixed_recip_DS.data_vars.values(), selected_attrs_dict)
for DA in tqdm(selected_DAs):
    # Slice data for selected q ranges (will need to rename q_xy if dimensions are differently named)
    sliced_DA = DA.sel(q_xy=slice(qxy_min, qxy_max), q_z=slice(qz_min, qz_max))
    qz_integrated_DA = sliced_DA.sum('q_xy')
    
    # Plot
    qz_integrated_DA.plot.line(label=DA.incident_angle)
    
plt.legend()
plt.grid(visible=True, which='major', axis='x')
plt.show()

In [ ]:
chi_min = 60
chi_max = None

selected_DAs = select_attrs(fixed_caked_DS.data_vars.values(), selected_attrs_dict)
for DA in tqdm(selected_DAs):
    # Slice dataarray to select plotting region 
    sliced_DA = DA.sel(chi=slice(chi_min,chi_max))
    
    # real_min = float(DA.sel(q_xy=slice(-0.5, -0.1), q_z=slice(0.1, 0.4)).compute().quantile(1e-3))
    real_min = float(DA.compute().quantile(0.05))
    cmin = 1 if real_min < 1 else real_min
    
    # cmax = float(DA.sel(q_xy=slice(-0.5, -0.1), q_z=slice(0.1, 2)).compute().quantile(1))   
    cmax = float(DA.compute().quantile(0.999))  
    
    # Plot sliced dataarray
    ax = sliced_DA.plot.imshow(cmap=cmap, norm=plt.Normalize(cmin, 10), figsize=(5,4), interpolation='antialiased')  # plot, optional parameter interpolation='antialiased' for image smoothing
    ax.colorbar.set_label('Intensity [arb. units]', rotation=270, labelpad=15)  # set colorbar label & parameters 
    ax.axes.set(title=f'Polar Plot: {DA.material} {DA.solvent}, {float(DA.incident_angle[2:])}° Incidence',
                xlabel='q$_r$ [Å$^{-1}$]', ylabel='$\chi$ [°]')  # set title, axis labels, misc
    ax.figure.set(tight_layout=True, dpi=130)  # Adjust figure dpi & plotting style
    
    plt.show()  # Comment to mute plotting output
    
    # Uncomment below line and set savepath/savename for saving plots, I usually like to check 
    # ax.figure.savefig(outPath.joinpath('PM6-Y6set_waxs', f'polar-2D_{DA.sample_id}_{chi_min}to{chi_max}chi_{DA.incident_angle}.png'), dpi=150)
    plt.close('all')